## Determinante

Tal y como ya hemos visto en clase, la variedad de herramientas proporcionadas por el algebra lineal son cruciales para desarrollar y fundamentar las bases de una variedad de tecnicas relacionadas con el aprendizaje automatico. Con ella, podemos describir el proceso de propagacion hacia adelante en una red neuronal, identificar mınimos locales en funciones multivariables (crucial para el proceso de retropropagacion) o la descripcion y empleo de metodos de reduccion de la dimensionalidad, como el analisis de componentes principales (PCA), entre muchas otras aplicaciones.

Cuando trabajamos en la practica dentro de este ambito, la cantidad de datos que manejamos puede ser muy grande, por lo que es especialmente importante emplear algoritmos eficientes y optimizados para reducir el coste computacional en la medida de lo posible. Por todo ello, el objetivo de este ejercicio es el de ilustrar las diferentes alternativas que pueden existir para realizar un proceso relacionado con el algebra lineal y el impacto que puede tener cada variante en terminos del coste computacional del mismo. En este caso en particular, y a modo de ilustracion, nos centraremos en el calculo del determinante de una matriz.


a) [1 punto] Implementa una funcion, `determinante_recursivo`, que obtenga el determinante de una matriz cuadrada utilizando la definicion recursiva de Laplace.


In [ ]:
# data analysis libs
import numpy as np
import pandas as pd

# utils libs
import copy
import random
import math


# graphical libs
import plotly.express as px
import plotly.graph_objects as go

# typehint libs
from typing import Callable
from numpy.typing import NDArray

# Avoid annoying warning for deprecations ...
import warnings


warnings.simplefilter(action="ignore", category=FutureWarning)

In [ ]:
def get_square_matrix(n: int, n_range: tuple[int, int] = (0, 100)) -> list[list]:
    """generate random matrix of n size

    Args:
        n (int): amount of rows/columns
        n_range (tuple[int, int], optional): range of numbers for each item in the matrix. Defaults to (0, 100).

    Returns:
        list[list]: generated matrix
    """
    return [[random.randint(*n_range) for _ in range(n)] for _ in range(n)]


def display_matrix(matrix: list[list]):
    """Display matrix in terminal nicely

    Args:
        matrix (list[list]): matrix that you want to display
    """
    print("")
    for i in range(len(matrix)):
        print(" ".join([f"{n:02}" for n in matrix[i]]))
    print("")


matrix = get_square_matrix(3)
assert len(matrix) == 3 and len(matrix[0]) == 3

display_matrix(matrix)

In [ ]:
def determinante_recursivo(matrix: list[list]) -> float:
    """get determinant of a matrix using Laplace recursive method

    Args:
        matrix (list[list]): squared matrix to obtain the determinant from

    Raises:
        Exception: in case the matrix is not squared

    Returns:
        float: value of the determinant
    """
    if len(matrix) != len(matrix[0]):
        raise Exception("Matriz no es cuadrada...")

    if len(matrix) == 2:
        a, b = matrix[0]
        c, d = matrix[1]
        return a * d - b * c

    res = 0
    for i, n in enumerate(matrix[0]):
        sub_matrix = [[el for j, el in enumerate(row) if j != i] for row in matrix[1:]]
        res += n * (-1) ** i * determinante_recursivo(sub_matrix)

    return res


matrix = get_square_matrix(4)

print("Matriz resultante:")
display_matrix(matrix)
print("Valor del determinante:", determinante_recursivo(matrix))

b) [0.5 puntos] Si A es una matriz cuadrada n×n y triangular (superior o inferior, es decir, con entradas nulas por debajo o por encima de la diagonal, respectivamente), ¿existe alguna forma de calcular de forma directa y sencilla su determinante? Justifıquese larespuesta.


In [ ]:
def is_valid_matrix_triangular(matrix: list[list]) -> bool:
    """checks if a matrix is triangular and squared

    Args:
        matrix (list[list]): matrix to check

    Returns:
        bool: if the matrix is triangular
    """
    is_superior = True
    is_inferior = True

    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if j > i and not math.isclose(matrix[i][j], 0, abs_tol=1e-12):
                is_superior = False

    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if j < i and not math.isclose(matrix[i][j], 0, abs_tol=1e-12):
                is_inferior = False

    return is_superior or is_inferior


def get_triangular_matrix(
    n: int, n_range: tuple[int, int] = (0, 100), superior=True
) -> list[list]:
    """generate a triangular matrix either superior or inferior

    Args:
        n (int): amount of rows/columns
        n_range (tuple[int, int], optional): range of numbers for each item in the matrix. Defaults to (0, 100).
        superior (bool, optional): if the resulting matrix should be superior or not. Defaults to True.

    Returns:
        list[list]: triangular matrix
    """
    matrix = get_square_matrix(n, n_range)

    if superior:
        for i in range(len(matrix)):
            for j in range(len(matrix)):
                if j > i:
                    matrix[i][j] = 0
    else:
        for i in range(len(matrix)):
            for j in range(len(matrix)):
                if j < i:
                    matrix[i][j] = 0

    return matrix


superior_matrix = get_triangular_matrix(4, superior=True)
inferior_matrix = get_triangular_matrix(4, superior=True)

print("Matriz Triangular superior:")
assert is_valid_matrix_triangular(superior_matrix)
display_matrix(superior_matrix)


print("Matriz Triangular inferior:")
assert is_valid_matrix_triangular(inferior_matrix)
display_matrix(inferior_matrix)

In [ ]:
def determinante_matriz_triangular(matrix: list[list]) -> float:
    """calculate determinant of a matrix using only the diagonal of it
    Args:
        matrix (list[list]): squared and diagonal matrix to obtain the determinant from

    Raises:
        Exception: in case the matrix is not diagonal

    Returns:
        float: value of the determinant
    """
    if not is_valid_matrix_triangular(matrix):
        raise Exception("Matriz no es triangular (inferior o superior)")

    res = matrix[0][0]
    for i in range(1, len(matrix)):
        res *= matrix[i][i]

    return res


# generate two triangular matrixes, one inferior and another superior to run the assertions
for superior in range(2):
    triangular_matrix = get_triangular_matrix(3, superior=superior)

    print("Matriz", "superior:" if superior else "inferior:")
    display_matrix(triangular_matrix)

    det_tri = determinante_matriz_triangular(triangular_matrix)
    det_rec = determinante_matriz_triangular(triangular_matrix)

    assert det_tri == det_rec
    print("Valor del determinante utlizando diagonal principal:", det_tri)
    print("Valor del determinante utlizando Laplace:", det_rec)


print(
    """\nJustificacion:
    
    El determinante para una matrix triangular se puede calcular como la multiplicacion de los elementos de su diagonal.
    Como se puede observar para las dos matrices triangulares (inferior o superior), el resultado del determinante mediante los dos metodos es el mismo.
    """
)

c) [0.5 puntos] Determınese de forma justificada como alteran el determinante de una matriz n × n las dos operaciones elementales siguientes:

- Intercambiar una fila (o columna) por otra fila (o columna).
- Sumar a una fila (o columna) otra fila (o columna) multiplicada por un escalar α.


In [ ]:
def swap_matrix_rows(matrix: list[list], rows: tuple[int, int]) -> list[list]:
    """swap rows of a matrix

    Args:
        matrix (list[list]): matrix
        row1 (tuple[int, int]): rows to swap

    Returns:
        list[list]: matrix with the rows swapped
    """
    # copy parameter matrix to avoid modifying parameters
    matrix_copy = copy.deepcopy(matrix)

    matrix_copy[rows[0]], matrix_copy[rows[1]] = (
        matrix_copy[rows[1]],
        matrix_copy[rows[0]],
    )

    return matrix_copy


def swap_matrix_columns(matrix: list[list], cols: tuple[int, int]) -> list[list]:
    """swap columns of a matrix

    Args:
        matrix (list[list]): matrix
        cols (tuple[int, int]): columns to swap

    Returns:
        list[list]: matrix with the rows swapped
    """
    # copy parameter matrix to avoid modifying parameters
    matrix_copy = copy.deepcopy(matrix)

    for row in matrix_copy:
        row[cols[0]], row[cols[1]] = row[cols[1]], row[cols[0]]

    return matrix_copy


matrix = [[1, 2], [3, 4]]

assert swap_matrix_rows(matrix, (0, 1)) == [[3, 4], [1, 2]]
assert swap_matrix_columns(matrix, (0, 1)) == [[2, 1], [4, 3]]

In [ ]:
original_matrix = get_square_matrix(3)

print("Matriz original: ")
display_matrix(original_matrix)
print("Valor del determinante:", determinante_recursivo(original_matrix))

matrix_with_exchange_row = swap_matrix_rows(original_matrix, (0, 1))
print("Matriz con la primera fila intercambiada con la segunda: ")
display_matrix(matrix_with_exchange_row)
print("Valor del determinante:", determinante_recursivo(matrix_with_exchange_row))


matrix_with_exchange_column = swap_matrix_columns(original_matrix, (0, 1))
print("Matriz con la primera columna intercambiada con la segunda: ")
display_matrix(matrix_with_exchange_column)
print("Valor del determinante:", determinante_recursivo(matrix_with_exchange_column))

print(
    """\nJustificacion:
    
    Al intercambiar una columna o una fila de una matriz, el determinante sera igual al valor del determinante original pero multiplicado por -1.
    """
)

In [ ]:
def sum_rows_in_matrix(
    matrix: list[list], rows: tuple[int, int], multiplier: float
) -> list[list]:
    """sum one row to another multiplied by a number

    Args:
        matrix (list[list]): given matrix
        rows (tuple[int, int]): rows to apply the operation
        multiplier (float): value for the multiplication

    Returns:
        list[list]: resulting matrix
    """
    # copy parameter matrix to avoid modifying parameters
    matrix_copy = copy.deepcopy(matrix)

    for i in range(len(matrix_copy)):
        matrix_copy[rows[0]][i] += matrix_copy[rows[1]][i] * multiplier

    return matrix_copy


def sum_columns_in_matrix(
    matrix: list[list], cols: tuple[int, int], multiplier: float
) -> list[list]:
    """sum one column to another multiplied by a number

    Args:
        matrix (list[list]): given matrix
        cols (tuple[int, int]): cols to apply the operation
        multiplier (float): value for the multiplication

    Returns:
        list[list]: resulting matrix
    """
    # copy parameter matrix to avoid modifying parameters
    matrix_copy = copy.deepcopy(matrix)

    for i in range(len(matrix_copy)):
        matrix_copy[i][cols[0]] += matrix_copy[i][cols[1]] * multiplier

    return matrix_copy


matrix = [[1, 2], [3, 4]]

assert sum_rows_in_matrix(matrix, (0, 1), 2) == [[7, 10], [3, 4]]
assert sum_columns_in_matrix(matrix, (0, 1), 2) == [[5, 2], [11, 4]]

In [ ]:
original_matrix = get_square_matrix(3)

print("Matriz original: ")
display_matrix(original_matrix)
print("Valor del determinante:", determinante_recursivo(original_matrix))


matrix_with_multiplied_rows = sum_rows_in_matrix(original_matrix, (0, 1), 2)
print("Matriz con la segunda fila multiplicada por 2 y sumada a la primera fila:")
display_matrix(matrix_with_multiplied_rows)
print("Valor del determinante:", determinante_recursivo(matrix_with_multiplied_rows))


matrix_with_multiplied_column = sum_columns_in_matrix(original_matrix, (0, 1), 2)
print("Matriz con la segunda columna multiplicada por 2 y sumada a la primera columna:")
display_matrix(matrix_with_multiplied_column)
print("Valor del determinante:", determinante_recursivo(matrix_with_multiplied_column))


print(
    """\nJustificacion:
    
    Al sumar a una fila (o columna) otra fila (o columna) multiplicada por un escalar, el valor del determinante no se ve alterado
    """
)

d) [1 punto] Investiga sobre el metodo de eliminacion de Gauss con pivoteo parcial e implementalo para escalonar una matriz (es decir, convertirla en una matriz triangular inferior) a partir de las operaciones elementales descritas en el apartado anterior.


In [ ]:
def gaussian_elimination(matrix: list[list]) -> list[list]:
    """perform the gaussian elimination method for a given matrix

    Args:
        matrix (list[list]): _description_

    Returns:
        list[list]: _description_
    """
    matrix_copy = copy.deepcopy(matrix)

    for i in range(len(matrix_copy)):
        # find the proper row pivot to avoid zero division and rounded errors
        for j in range(i + 1, len(matrix_copy)):
            if matrix_copy[i][i] < matrix_copy[j][i]:
                matrix_copy = swap_matrix_rows(matrix_copy, (i, j))

        # apply rows operations in order to ensure triangularly of the matrix
        for j in range(i + 1, len(matrix_copy)):
            multiplier = matrix_copy[j][i] / matrix_copy[i][i]
            matrix_copy = sum_rows_in_matrix(matrix_copy, (j, i), -multiplier)

    return matrix_copy


# matrix = get_square_matrix(3)
matrix = [[1, 2, 3], [2, 4, 6], [1, 5, 6]]
print("Original matrix:")
display_matrix(matrix)

matrix_gaussian_elimination = gaussian_elimination(matrix)
print("Gaussian elimination matrix:")
display_matrix(matrix_gaussian_elimination)

e) [0.5 puntos] ¿Como se podrıa calcular el determinante de una matriz haciendo beneficio de la estrategia anterior y del efecto de aplicar las operaciones elementales pertinentes? Implementa una nueva funcion, `determinante_gauss`, que calcule el determinante de una matriz utilizando eliminacion gaussiana.


In [ ]:
def determinante_gauss(matrix: list[list]) -> float:
    """Get determinant of a matrix first by using the Gauss method to make the matrix triangular and then obtain the determinant of a triangular matrix

    Args:
        matrix (list[list]): initial matrix

    Returns:
        float: value of the determinant
    """
    matrix_gaussian_elimination = gaussian_elimination(matrix)
    return determinante_matriz_triangular(matrix_gaussian_elimination)


matrix = get_square_matrix(3)
det_matrix = determinante_recursivo(matrix)
det_matrix_gaussian_triangular = determinante_gauss(matrix)

print("Matriz original:")
display_matrix(matrix)

print("Valor del determinante utilizando Laplace:", det_matrix)
print(
    "Valor del determinante utilizando Gauss y Diagonal principal:",
    det_matrix_gaussian_triangular,
)


print(
    """\nJustificacion:
    
    Una vez hecho la diagonalizacion de una matrix utilizando Gauss, se puede obtener facilmente el determinante utilizando el metodo para matrices triangulares.
    Donde el determinante se puede calcular como la multiplicacion de los elementos de su diagonal.
    """
)

f ) [0.5 puntos] Obten la complejidad computacional asociada al calculo del determinante con la definicion recursiva y con el metodo de eliminacion de Gauss con pivoteo parcial.


In [ ]:
print(
    """Complejidad computacional para el calculo del determinante:

    - Definicion recursiva (Laplace): O(n!) por su propiedad recursiva
    - Definicion recursiva (Laplace): O(n**3) son 3 for anidados
    """
)

g) [1 punto] Utilizando `numpy.random.rand`, genera matrices cuadradas aleatorias de la forma $An ∈ R n×n$, para $2 ≤ n ≤ 10 $

Confecciona una tabla comparativa del tiempo de ejecucion asociado a cada una de las variantes siguientes, interpretando los resultados:

- Utilizando determinante recursivo.
- Empleando determinante gauss.
- Haciendo uso de la funcion preprogramada numpy.linalg.det.


In [ ]:
from timeit import timeit
import plotly.express as px
import pandas as pd

data = []

input_range = range(2, 11)
input_sizes = [np.random.rand(n, n).tolist() for n in input_range]

for n in input_sizes:
    recur_time = timeit(lambda: determinante_recursivo(n), number=1)
    gauss_time = timeit(lambda: determinante_gauss(n), number=1)
    numpy_time = timeit(lambda: np.linalg.det(n), number=1)

    data.extend(
        [
            ["Recursive", len(n), recur_time],
            ["Gauss", len(n), gauss_time],
            ["Numpy", len(n), numpy_time],
        ]
    )


df = pd.DataFrame(data, columns=["Method", "n", "Time"])

fig = px.line(
    df,
    x="n",
    y="Time",
    color="Method",
    markers=True,
    title="Time to calculate matrix determinant",
)

fig.show()

## Descenso del gradiente

En este ejercicio trabajaremos con el metodo de descenso de gradiente, el cual constituye otra herramienta crucial, en esta ocasion de la rama del calculo, para el proceso de retropropagacion asociado al entrenamiento de una red neuronal.


a) [1 punto] Programese en Python el metodo de descenso de gradiente para funciones de $n$ variables. La funcion debera tener como parametros de entradas:

- El gradiente de la funcion que se desea minimizar $∇f$ (puede venir dada como otra funcion previamente implementada, `grad_f`, con entrada un vector, representando el punto donde se quiere calcular el gradiente, y salida otro vector, representando el gradiente de $f$ en dicho punto).
- Un valor inicial $x0 ∈ Rn$ (almacenado en un vector de $n$ componentes).
- El ratio de aprendizaje $γ$ (que se asume constante para cada iteracion).
- Un parametro de tolerancia `tol` (con el que finalizar el proceso cuando $∥∇f(x)∥2$ < tol).
- Un numero maximo de iteraciones `maxit` (con el fin de evitar ejecuciones indefinidas en caso de divergencia o convergencia muy lenta).

La salida de la funcion debera ser la aproximacion del $x$ que cumple $f′(x) ≈ 0$, correspondiente a la ultima iteracion realizada en el metodo.


In [ ]:
def gradient_descent(
    grad_f: Callable,
    initial_point: NDArray,
    learning_rate=0.01,
    maxit=1000,
    tol=1e-6,
) -> list[NDArray]:
    """get minimum point of a function using method of gradient descent with derivate

    Args:
        grad_f (Callable): derivate of the function
        initial_point (NDArray): start point to start looking for the minimum point
        learning_rate (float, optional): learning rate to move the point on each iteration. Defaults to 0.01.
        maxit (int, optional): numbers of max iterations to run. Defaults to 1000.
        tol (float, optional): tolerance value to finish the algorithm in case we found point of convergence. Defaults to 1e-6.

    Returns:
        list[NDArray]: history of the points inside the algorithm of gradient descent
    """
    point = np.array(initial_point)
    history = np.array([point])

    for _ in range(maxit):
        # get gradient from the derivate function
        grad = np.array(grad_f(*point))

        # Check for point convergence
        if np.linalg.norm(grad) < tol:
            break

        # get new point for the next iteration
        point -= grad * learning_rate

        # save it in history
        history = np.concatenate((history, [point]), axis=0)

    return history

In [ ]:
def draw_function_2d_with_gradient(
    f: Callable, graph_range: tuple[float, float], descent_history: NDArray
):
    """Create plot using plotly to represent a gradient descent result of function

    Args:
        f (Callable): function to render
        graph_range (tuple[float, float]): range to draw the function
        descent_history (NDArray): history of the gradient descent algorithm
    """
    # Vectorize function in order to be possible to support numpy array operations
    f_vector = np.vectorize(f)

    x = np.linspace(*graph_range, 100)

    fig = go.Figure()

    fig.add_trace(go.Scatter(x=x, y=f_vector(x), mode="lines", name="Function"))

    descent_df = pd.DataFrame(
        [(*p, f_vector(*p)) for p in descent_history], columns=["x", "y"]
    )

    try:
        fig.add_trace(
            go.Scatter(
                x=descent_df["x"],
                y=descent_df["y"],
                mode="markers",
                name="Gradient",
            )
        )
    except OverflowError:
        print("Cannot draw gradient history due to Overflow Error")

    fig.update_layout(title=f"Punto minimo encontrado en {descent_history[-1]}")

    fig.show()

b) Sea la funcion $f : R → R$ dada por: $$f(x) = 3x^4 + 4x^3 − 12x^2 + 7$$


In [ ]:
def f(x: float) -> float:
    return 3 * (x**4) + 4 * (x**3) - 12 * x**2 + 7


def df(x: float) -> float:
    return 12 * (x**3) + 12 * (x**2) - 12 * x

I. [0.5 puntos] Aplica el metodo sobre $f(x)$ con $x0$= 3 $γ$= 0.001, `tol`=1e-12, `maxit`=1e5.


In [ ]:
descent_history = gradient_descent(
    df,
    initial_point=[float(3)],
    learning_rate=0.001,
    tol=1e-12,
    maxit=int(1e5),
)


draw_function_2d_with_gradient(f, (-3, 3), descent_history)

II. [0.5 puntos] Aplica de nuevo el metodo sobre $f(x)$ con $x0$ = 3, $γ$ = 0.01, `tol`=1e-12, `maxit`=1e5.


In [ ]:
descent_history = gradient_descent(
    df,
    initial_point=[float(3)],
    learning_rate=0.01,
    tol=1e-12,
    maxit=int(1e5),
)

draw_function_2d_with_gradient(f, (-3, 3), descent_history)

III. [0.5 puntos] Contrasta e interpreta los dos resultados obtenidos en los apartados anteriores y comparalos con los mınimos locales obtenidos analıticamente. ¿Que influencia puede llegar a tener la eleccion del ratio de aprendizaje $γ$?


In [ ]:
print(
    """Justificacion:
      La tasa de aprendizaje (y) tiene una influencia significativa en el comportamiento del descenso del gradiente:
        - Si es demasiado alta puede causar oscilaciones o divergencia.
        - Si es demasiado baja puede resultar en una convergencia muy lenta.
        - Dependiendo del punto inicial, puede converger a diferentes minimos locales en funciones no convexas.
      """
)

IV. [0.5 puntos] Aplica nuevamente el metodo sobre $f(x)$ con $x0$ = 3, $γ$ = 0.1, `tol`=1e-12, `maxit`=1e5. Interpreta el resultado.


In [ ]:
descent_history = gradient_descent(
    df,
    initial_point=[float(3)],
    learning_rate=0.1,
    tol=1e-12,
    maxit=int(1e5),
)

print(
    """Justificacion:
      
      Al tener un ratio de aprendizaje tan grande el algoritmo no puedo llegar a encontrar un punto de divergencia y por lo tanto tiende al infinito
    """
)

draw_function_2d_with_gradient(f, (-3, 3), descent_history)

V. [0.5 puntos] Finalmente, aplica el metodo sobre $f(x)$ con $x0$ = 0, $γ$ = 0.001, `tol`=1e-12, `maxit`=1e5. Interpreta el resultado y comparalo con el estudio analıtico de $f$. ¿Se trata de un resultado deseable? ¿Por que? ¿A que se debe este fenomeno?


In [ ]:
descent_history = gradient_descent(
    df,
    initial_point=[float(0)],
    learning_rate=0.0001,
    tol=1e-12,
    maxit=int(1e5),
)

print(
    """Justificacion:
      
      En el punto inicial seleccionado no es posible convergir a ningun otro punto, dado que como se puede observar la funcion no cuenta con una pendiente definida en ese punto. 
      Es decir la derivada de la funcion en este punto es 0, y por lo tanto el algoritmo para.
    """
)

draw_function_2d_with_gradient(f, (-3, 3), descent_history)

c) Sea la funcion $g : R^2 -> R^2$ dada por $$g(x, y) = x^2 + y^3 + 3xy + 1$$


In [ ]:
def f(x: float, y: float) -> float:
    return x**2 + y**3 + 3 * x * y + 1


def df(x: float, y: float) -> list:
    return [2 * x + 3 * y, 3 * y**2 + 3 * x]

In [ ]:
def draw_function_3d_with_gradient(
    f: Callable, graph_range: tuple[float, float], descent_history: NDArray
):
    """Create plot using plotly to represent a gradient descent result of function

    Args:
        f (Callable): function to render
        graph_range (tuple[float, float]): range to draw the function
        descent_history (NDArray): history of the gradient descent algorithm
    """
    # Vectorize function in order to be possible to support numpy array operations
    f_vector = np.vectorize(f)

    # Create a grid of x and y values
    x, y = np.meshgrid(np.linspace(*graph_range, 100), np.linspace(*graph_range, 100))
    z = f_vector(x, y)

    # Create the plot
    fig = go.Figure(data=[go.Surface(z=z, x=x, y=y, opacity=0.7)])

    # convert history to pandas df
    descent_df = pd.DataFrame(
        [(*p, f_vector(*p)) for p in descent_history], columns=["x", "y", "z"]
    )

    # mark start and end point of the gradient history
    points_df = pd.concat([descent_df.iloc[[0]], descent_df.iloc[[-1]]])
    fig.add_traces(
        go.Scatter3d(
            x=points_df["x"],
            y=points_df["y"],
            z=points_df["z"],
            mode="markers+text",
            text=["Start", "End"],
        )
    )

    # add gradient line
    fig.add_traces(
        px.line_3d(
            descent_df, x="x", y="y", z="z", color_discrete_sequence=["white"]
        ).data
    )

    fig.update_layout(title=f"Punto minimo encontrado en {descent_history[-1]}")

    # Show the plot
    fig.show()

I. [0.5 puntos] Aplıquese el metodo sobre $g(x, y)$ con $x0$ = (−1, 1), $γ$ = 0.01, `tol`=1e-12, `maxit`=1e5.


In [ ]:
descent_history = gradient_descent(
    df,
    initial_point=[float(-1), float(1)],
    learning_rate=0.01,
    tol=1e-12,
    maxit=int(1e5),
)

draw_function_3d_with_gradient(f, (-5, 5), descent_history)

II [0.5 puntos] ¿Que ocurre si ahora partimos de $x0 = (0, 0)$? ¿Se obtiene un resultado deseable?


In [ ]:
descent_history = gradient_descent(
    df,
    initial_point=[float(0), float(0)],
    learning_rate=0.01,
    tol=1e-12,
    maxit=int(1e5),
)

print(
    """Justificacion:
      
      En el punto inicial seleccionado no es posible convergir a ningun otro punto, dado que como se puede observar la funcion no cuenta con una pendiente definida en ese punto. 
      Es decir la derivada de la funcion en este punto es 0, y por lo tanto el algoritmo para.
    """
)

draw_function_3d_with_gradient(f, (-5, 5), descent_history)

III. [0.5 puntos] Realicese el estudio analıtico de la funcion y utilıcese para explicar y contrastar los resultados obtenidos en los dos apartados anteriores.
